In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
import numpy as np

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [98]:
def load_data():
    movie_dict = defaultdict(dict)
    count = 0
    with open('data_cbf/content_movie_list.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1
            else:
                count += 1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]

    return(movie_dict)

In [86]:
def print_pred_movies(y_p, item, movie_dict, maxcount=10):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     movie_dict[movie_id]['title'], movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

In [85]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

In [8]:
def make_equal_length(df1, df2, df3):
    # Find the largest dataframe based on the number of rows
    max_rows = max(len(df1), len(df2), len(df3))
    
    # Duplicate rows of the dataframes until they all have the same length
    df1_new = pd.concat([df1] * ((max_rows + len(df1) - 1) // len(df1)), ignore_index=True).head(max_rows)
    df2_new = pd.concat([df2] * ((max_rows + len(df2) - 1) // len(df2)), ignore_index=True).head(max_rows)
    df3_new = pd.concat([df3] * ((max_rows + len(df3) - 1) // len(df3)), ignore_index=True).head(max_rows)
    
    return df1_new, df2_new, df3_new

# DATA 

In [64]:
item_data=pd.read_csv("data_cbf/content_item_train.csv")

In [65]:
item_data.head()

,6874,2003,3.9618320610687023,1,0,0.1,0.2,0.3,1.1,0.4,0.5,0.6,0.7,0.8,0.9,0.10,1.2
0,8798,2004,3.761364,1,0,0,0,0,1,0,1,0,0,0,0,0,1
1,46970,2006,3.250000,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,48516,2006,4.252336,0,0,0,0,0,1,0,1,0,0,0,0,0,1
3,58559,2008,4.238255,1,0,0,0,0,1,0,1,0,0,0,0,0,0
4,60756,2008,3.553571,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [18]:
ratings_data=pd.read_csv("data_cbf/content_user_train.csv")

In [22]:
df_users=ratings_data

In [23]:
df_users.head()

,2.0,22.0,4.0,3.95,4.25,0.0,0.0.1,4.0.1,4.12,4.0.2,4.04,0.0.2,3.0,4.0.3,0.0.3,3.88,3.89
0,2.0,22.0,4.0,3.95,4.25,0.0,0.0,4.0,4.12,4.0,4.04,0.0,3.0,4.0,0.0,3.88,3.89
1,2.0,22.0,4.0,3.95,4.25,0.0,0.0,4.0,4.12,4.0,4.04,0.0,3.0,4.0,0.0,3.88,3.89
2,2.0,22.0,4.0,3.95,4.25,0.0,0.0,4.0,4.12,4.0,4.04,0.0,3.0,4.0,0.0,3.88,3.89
3,2.0,22.0,4.0,3.95,4.25,0.0,0.0,4.0,4.12,4.0,4.04,0.0,3.0,4.0,0.0,3.88,3.89
4,2.0,22.0,4.0,3.95,4.25,0.0,0.0,4.0,4.12,4.0,4.04,0.0,3.0,4.0,0.0,3.88,3.89


In [25]:
y=pd.read_csv("data_cbf/content_y_train.csv")

In [26]:
y_train=y

In [27]:
y_train.shape

(50883, 1)

In [28]:
df_movies,df_users,y_train=make_equal_length(item_data,df_users,pd.DataFrame(y_train))

In [30]:
item_train=df_movies.values

In [31]:
user_train=df_users.values

In [33]:
num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time

In [34]:
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, item

In [35]:
y_train=y_train.values

In [36]:
item_train.shape

(50883, 17)

In [37]:
user_train.shape

(50883, 17)

In [38]:
y_train.shape

(50883, 1)

In [40]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [41]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")


movie/item training data shape: (40706, 17)
movie/item test data shape: (10177, 17)


In [42]:
print(f"movie/item training data shape: {user_train.shape}")
print(f"movie/item test data shape: {user_test.shape}")

movie/item training data shape: (40706, 17)
movie/item test data shape: (10177, 17)


In [43]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    tf.keras.layers.Dense(256,activation="relu"),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dense(num_outputs)
  
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    tf.keras.layers.Dense(256,activation="relu"),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dense(num_outputs)
  
  
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 sequential (Sequential)     (None, 32)                   40864     ['input_1[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 32)                   41376     ['input_2[0][0]']             
                                                                                              

In [44]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [45]:
user_train.shape

(40706, 17)

In [46]:
item_train.shape

(40706, 17)

In [47]:
y_train.shape

(40706, 1)

In [49]:
# Disable eager execution
tf.config.run_functions_eagerly(True)
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30


C:\Users\DELL\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1273/1273 [==============================] - 69s 54ms/step - loss: 0.1211
Epoch 2/30
1273/1273 [==============================] - 61s 48ms/step - loss: 0.1101
Epoch 3/30
1273/1273 [==============================] - 52s 41ms/step - loss: 0.1054
Epoch 4/30
1273/1273 [==============================] - 50s 39ms/step - loss: 0.1022
Epoch 5/30
1273/1273 [==============================] - 50s 39ms/step - loss: 0.0989
Epoch 6/30
1273/1273 [==============================] - 50s 39ms/step - loss: 0.0966
Epoch 7/30
1273/1273 [==============================] - 56s 44ms/step - loss: 0.0938
Epoch 8/30
1273/1273 [==============================] - 50s 39ms/step - loss: 0.0917
Epoch 9/30
1273/1273 [==============================] - 50s 39ms/step - loss: 0.0897
Epoch 10/30
1273/1273 [==============================] - 51s 40ms/step - loss: 0.0882
Epoch 11/30
1273/1273 [==============================] - 51s 40ms/step - loss: 0.0867
Epoch 12/30
1273/1273 [==============================] - 61s 48ms/step - l

In [52]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 3s 10ms/step - loss: 0.0843


0.08427710831165314

In [51]:
model.save_weights('model_weights_content_based_complete.h5')

In [70]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [107]:
from numpy import genfromtxt
item_vecs = genfromtxt('data_cbf/content_item_vecs.csv', delimiter=',')

In [108]:
from collections import defaultdict
import csv
movie_dict=load_data()

In [109]:
import tabulate

In [110]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

 8/27 [=======>......................] - ETA: 0s

C:\Users\DELL\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


27/27 [==============================] - 0s 7ms/step


y_p,movie id,rating ave,title,genres
4.1,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.1,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.1,59501,3.5,"Chronicles of Narnia: Prince Caspian, The (2008)",Adventure|Children|Fantasy
4,41566,3.4,"Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The (2005)",Adventure|Children|Fantasy
3.9,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
3.9,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
3.9,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
3.8,4896,3.8,Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),Adventure|Children|Fantasy
3.8,59387,4,"Fall, The (2006)",Adventure|Drama|Fantasy
3.8,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
